<a href="https://colab.research.google.com/github/gcosma/DeepLearningTutorials/blob/master/SimpleSequentialModelwKFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tutorial 2: HOW TO Create your first Sequential Model in Keras: With k-fold cross validation and a confusion matrix** 

by Dr Georgina Cosma

This tutorial is based on  

https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

09 August 2019

Learning outcomes:
* Load Data.
* Define a Sequential model with k-fold cross validation. 
* Compile Model.
* Evaluate Model and display confusion matrix.


**Not using Colab?** If you are not using Colab you will need to Setup a Python Environment for 
Machine Learning and Deep Learning with Anaconda. You must have Python 2 or 3 
installed and configured. You must install SciPy (including NumPy) and the relevant
libraries including Keras. 

**Using Colab:** Some difficulties may be
experienced with mounting, but the code and explanation here will help you overcome these. 



**Step 1: Import libraries** 

In [0]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


**Step 2: Mount to Google Drive in order to access your data file**

In [0]:
from google.colab import drive
drive.mount('/content/drive')
#!ls "/content/drive/My Drive/Colab Notebooks"

#if you need to remount
#drive.mount("/content/drive", force_remount=True)

#If you want to unmount and reset then: 
#Step 1: From the menu select Runtime--->Reset all Runtimes... 
#Step 2: Runtime--->Run all or you can run each Cell at a time. There will be a message 
# "Go to a URL in a browser" and you must click on that and copy and paste the authorisation code 
# from the page into the authorisation code text box.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Step 3: Load the dataset.**

In [0]:
# load the dataset
dataset = np.loadtxt("/content/drive/My Drive/Colab Notebooks/Tutorials/TutorialData/pima.txt", delimiter=",")
print(dataset)

[[  6.    148.     72.    ...   0.627  50.      1.   ]
 [  1.     85.     66.    ...   0.351  31.      0.   ]
 [  8.    183.     64.    ...   0.672  32.      1.   ]
 ...
 [  5.    121.     72.    ...   0.245  30.      0.   ]
 [  1.    126.     60.    ...   0.349  47.      1.   ]
 [  1.     93.     70.    ...   0.315  23.      0.   ]]


**Step 4: Split input (X) and output (Y) variables into separate matrices.**

* **Dataset contains:** 
* **Number of input variables: 8.          Columns 0 to 7**   
* **Number of output variables: 1.        Column 8**

* **Let X be the m x n feature by case matrix. In this example a case is a person's data, a.k.a. record.**
* **Let Y be a 1 x n vector holding all the labels. One row in Y corresponds to a row in X. So every person's record is assigned a label.**

**Dataset has 9 columns. Range 0:8 will select columns from 0 to 7, stopping before index 8.**

In [0]:
#Split into X and Y variables
#Input data and labels need to be separated before training the model
X = dataset[0:760,0:8]      #select rows 0 to 759 and columns 0 to 7. Columns 0 to 7 hold the input data.
Y = dataset[0:760,8]        #select rows 0 to 759 and column 8. Column 8 is the last column holding all the labels.

#The last 8 records for testing the model  (record 760 to 767)
X1 = dataset[760:768,0:8]   #select rows 760 to 767 and columns 0 to 7. Columns 0 to 7 hold the input data.
Y1 = dataset[760:768,8]     #select rows 760 to 767 and column 8. Column 8 is the last column holding all the labels.

#check the size of matrix X
#dataset.shape
X.shape
X1.shape

#print a record of dataset X and X1. 
#print(X[759]) #print record 759
#print(X1[0]) #print record 0

(8, 8)

**Step 5: Create the Sequential model**
A Sequential model is a linear stack of layers.
Code for kfold was taken from https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

fit(X,Y) is for training the model with the given inputs X (and corresponding training labels Y). Adjust the number of epochs to avoid overtraining your network.

evaluate(X,Y) is for evaluating the already trained model using the training data X and the corresponding labels Y. Returns the loss value and metrics values for the model. We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset. This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data.


In [0]:

# define 10-fold cross validation test harness
#numpy.random.seed(7)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
cvscores = []
for train, test in kfold.split(X, Y):
  # #First layer in the Sequential model
	model = Sequential()
  #Dense(n) is a fully-connected layer with x hidden units.  
  #In the first layer, you must specify the expected input data shape: input_dim=8 dimensional vectors.

  # Line model.add(Dense(12, input_dim=8, activation='relu', name='input_layer')), does the following:
  #defines the input layer as having 8 inputs.
  #defines a hidden layer with 12 neurons, connected to the input layer that use relu activation function.
  #initializes all weights using a sample of uniform random numbers.
	model.add(Dense(12, input_dim=8, activation='relu'))
  # now the model will take as input arrays of shape (*, 8)
  # and output arrays of shape (*, 12)

  # after the first layer, you don't need to specify
  # the size of the input anymore.
	model.add(Dense(8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model: Compile defines the loss function, the optimizer and the metrics.
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model: Fit the model. fit(X,Y) is for training the model with the given inputs X 
  # (and corresponding training labels Y). Adjust the number of epochs to avoid overtraining your network.
	model.fit(X[train], Y[train], epochs=150, batch_size=10, verbose=0)
	# evaluate the model
	scores = model.evaluate(X[test], Y[test], verbose=0)
	print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
	cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

acc: 64.94%
acc: 81.82%
acc: 72.73%
acc: 66.23%
acc: 64.47%
acc: 78.95%
acc: 65.33%
acc: 78.67%
acc: 70.67%
acc: 65.33%
70.91% (+/- 6.39%)


**Step 6: Predict on previously unseen data. The predict(X1) method is used for making the actual prediction using the trained model. It generates output predictions for the input samples in X1. Here we are using the predict(X1) function predict the labels of dataset X1 (test dataset).**

In [0]:
# calculate predictions
#X1 data was not used to train or validate the model. Here we test the model using new data, X1. 
predictions = model.predict(X1)
# round predictions
rounded = [round(x1[0]) for x1 in predictions]
print(rounded)

#Simple confusion matrix using sklearn.metrics 
results = confusion_matrix(Y1, rounded)
print(results)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
({0.0, 1.0}, [[6, 2], [0, 0]])


**Here is the code for creating your own confusion matrix.**
The code was taken from https://machinelearningmastery.com/implement-machine-learning-algorithm-performance-metrics-scratch-python/

In [0]:
# Example of Calculating and Displaying a better looking Confusion Matrix

#def defines a function for calculating the confusion matrix
# calculate a confusion matrix
def confusion_matrix(actual, predicted):
	unique = set(actual)
	matrix = [list() for x in range(len(unique))]
	for i in range(len(unique)):
		matrix[i] = [0 for x in range(len(unique))]
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for i in range(len(actual)):
		x = lookup[actual[i]]
		y = lookup[predicted[i]]
		matrix[y][x] += 1
	return unique, matrix
 
# pretty print a confusion matrix
def print_confusion_matrix(unique, matrix):
	print('(A)' + ' '.join(str(x) for x in unique))
	print('(P)-------')
	for i, x in enumerate(unique):
		print("%s| %s" % (x, ' '.join(str(x) for x in matrix[i])))
 

 **Step 7: Use the confusion_matrix function to print confusion matrices for the  Predict step (Step 9). X1 has  8 records.**

In [0]:
# Test confusion matrix for the Evaluate results
unique, matrix = confusion_matrix(Y1, rounded)
print_confusion_matrix(unique, matrix)



(A)0.0 1.0
(P)-------
0.0| 6 2
1.0| 0 0


In [0]:
# Print the Sequential model structure
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_79 (Dense)             (None, 12)                108       
_________________________________________________________________
dense_80 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_81 (Dense)             (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
None


**Step 8: How to install the GraphViz Library  http://www.graphviz.org/**



In [0]:
#!pip install -q pydot
from keras.utils.vis_utils import plot_model
#Prints the model to a file in the directory
plot_model(model, to_file='/content/drive/My Drive/Colab Notebooks/Tutorials/model_plot3.png', show_shapes=True, show_layer_names=True)

**Further Reading:**
About Colab: https://www.youtube.com/watch?v=inN8seMm7UI

**Examples to run (These are not in Colab):**
There are excellent examples in https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

https://keras.io/getting-started/sequential-model-guide/#examples

In the examples, you will also find example models for benchmark datasets:

* CIFAR10 small images classification: Convolutional Neural Network (CNN) with realtime data augmentation

* IMDB movie review sentiment classification: LSTM over sequences of words
Reuters newswires topic classification: Multilayer Perceptron (MLP)

* MNIST handwritten digits classification: MLP & CNN
Character-level text generation with LSTM




** *Many thanks to Jason Brownlee for providing excellent tutorials for machine learning and for providing permission to publish this tutorial. This tutorial is based on Jason Brownlee's Machine Learning Mastery tutorials https://machinelearningmastery.com   The website contains tutorials for Machine Learning.* ** 

